In [26]:
h2o.cluster().show_status()

H2O cluster uptime:,43 mins 38 secs
H2O cluster version:,3.16.0.1
H2O cluster version age:,6 days
H2O cluster name:,H2O_from_python_byungsangkim_fxfcq5
H2O cluster total nodes:,1
H2O cluster free memory:,825 Mb
H2O cluster total cores:,2
H2O cluster allowed cores:,2
H2O cluster status:,"locked, healthy"
H2O connection url:,http://127.0.0.1:54321
H2O connection proxy:,None


In [1]:
import pandas as pd
import numpy as np
import os
import xlrd
import urllib
import h2o
from h2o.grid.grid_search import H2OGridSearch

h2o.init(max_mem_size_GB = 2)

local_path = os.getcwd()
spamdata = h2o.import_file(os.path.realpath(local_path+'/datasets/spamdata.data'))

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_77"; Java(TM) SE Runtime Environment (build 1.8.0_77-b03); Java HotSpot(TM) 64-Bit Server VM (build 25.77-b03, mixed mode)
  Starting server from /Users/byungsagkim/Library/Python/3.6/lib/python/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/xz/jhfpyxt16fd4f_5_pwm1nzcm0000gn/T/tmpyp3592cn
  JVM stdout: /var/folders/xz/jhfpyxt16fd4f_5_pwm1nzcm0000gn/T/tmpyp3592cn/h2o_byungsangkim_started_from_python.out
  JVM stderr: /var/folders/xz/jhfpyxt16fd4f_5_pwm1nzcm0000gn/T/tmpyp3592cn/h2o_byungsangkim_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,05 secs
H2O cluster version:,3.16.0.1
H2O cluster version age:,6 days
H2O cluster name:,H2O_from_python_byungsangkim_z2ywfq
H2O cluster total nodes:,1
H2O cluster free memory:,1.766 Gb
H2O cluster total cores:,2
H2O cluster allowed cores:,2
H2O cluster status:,"accepting new members, healthy"
H2O connection url:,http://127.0.0.1:54321
H2O connection proxy:,None


Parse progress: |█████████████████████████████████████████████████████████| 100%


In [2]:
spamdata['C58'] = spamdata['C58'].asfactor()
train,valid,test = spamdata.split_frame([0.6,.2], seed=1234)
spam_X=spamdata.col_names[:-1]
spam_Y=spamdata.col_names[-1]

## Grid search for getting optimal params

In [39]:
from h2o.estimators import H2ORandomForestEstimator 

hyper_parameters={'ntrees':[300], 'max_depth':[3,6,10,12,50], \
                  'balance_classes':['True', 'False'], 'sample_rate':[.5,.6,.8,.9]}
grid_search = H2OGridSearch(model = H2ORandomForestEstimator, hyper_params=hyper_parameters)
grid_search.train(x=spam_X, y=spam_Y, training_frame=train)
print('This is the optimum solution for hyper parameters search %s' % grid_search.show())


drf Grid Build progress: |████████████████████████████████████████████████| 100%
     balance_classes max_depth ntrees sample_rate  \
0               true        50    300         0.9   
1               true        50    300         0.8   
2               true        50    300         0.6   
3               true        50    300         0.5   
4               true        12    300         0.9   
5               true        12    300         0.8   
6               true        12    300         0.6   
7               true        12    300         0.5   
8              false        50    300         0.8   
9              false        12    300         0.8   
10             false        12    300         0.6   
11             false        50    300         0.6   
12             false        12    300         0.9   
13              true        10    300         0.9   
14             false        50    300         0.5   
15             false        12    300         0.5   
16              tr

## Using DRF

In [40]:
final = H2ORandomForestEstimator(ntrees=300, max_depth=50, balance_classes=True, sample_rate=.9)
final.train(x=spam_X, y=spam_Y, training_frame=train)
print(final.predict(test))

drf Model Build progress: |███████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%


predict,p0,p1
1,0.00973471,0.990265
1,0.00719774,0.992802
1,0.121163,0.878837
1,0.0521552,0.947845
1,0.0166704,0.98333
1,0.00260096,0.997399
0,0.773304,0.226696
1,0.000274015,0.999726
1,0.0927196,0.90728
1,0.00519705,0.994803


## Save and Load the Model

In [42]:
# save the model
model_path = h2o.save_model(model=final, path=os.getcwd() + "/model/cart", force=True)

print(model_path)

# load the model
saved_cart = h2o.load_model(model_path)
print(saved_cart.predict(test))

/Users/byungsagkim/opt/python/model/cart/DRF_model_python_1512024223931_4062
drf prediction progress: |████████████████████████████████████████████████| 100%


predict,p0,p1
1,0.00973471,0.990265
1,0.00719774,0.992802
1,0.121163,0.878837
1,0.0521552,0.947845
1,0.0166704,0.98333
1,0.00260096,0.997399
0,0.773304,0.226696
1,0.000274015,0.999726
1,0.0927196,0.90728
1,0.00519705,0.994803


## Using GBM

In [45]:
from h2o.estimators import H2OGradientBoostingEstimator

hyper_parameters = {'ntrees':[300], 'max_depth':[12,30,50], 'sample_rate':[.5,.7,1], \
                   'col_sample_rate':[.9,1], 'learn_rate':[0.1,.1,.3],}
grid_search = H2OGridSearch(H2OGradientBoostingEstimator, hyper_parameters)
grid_search.train(x=spam_X, y=spam_Y, training_frame=train)
print('this is the optimum solution for hyper parameters search %s' % grid_search.show())

gbm Grid Build progress: |████████████████████████████████████████████████| 100%
     col_sample_rate learn_rate max_depth ntrees sample_rate  \
0                0.9        0.3        30    300         0.7   
1                1.0        0.3        50    300         0.7   
2                0.9        0.3        50    300         0.7   
3                0.9        0.3        12    300         0.7   
4                1.0        0.3        12    300         0.7   
5                0.9        0.3        30    300         0.5   
6                1.0        0.3        30    300         0.7   
7                0.9        0.3        12    300         0.5   
8                1.0        0.3        30    300         0.5   
9                1.0        0.3        12    300         0.5   
10               1.0        0.3        50    300         0.5   
11               0.9        0.1        30    300         0.7   
12               0.9        0.1        50    300         0.7   
13               0.9   

In [4]:
from h2o.estimators import H2OGradientBoostingEstimator

spam_gbm2 = H2OGradientBoostingEstimator(ntrees=300, learn_rate=0.3, max_depth=30, \
                                         sample_rate=0.7, col_sample_rate=0.9, \
                                         score_each_iteration=True, seed=2000000)
spam_gbm2.train(spam_X, spam_Y, training_frame=train, validation_frame=valid)
confusion_matrix = spam_gbm2.confusion_matrix(metrics="accuracy")
print(confusion_matrix)

gbm Model Build progress: |███████████████████████████████████████████████| 100%
Confusion Matrix (Act/Pred) for max accuracy @ threshold = 0.9935320164798572: 


,0,1,Error,Rate
0,1683.0,0.0,0.0,(0.0/1683.0)
1,3.0,1077.0,0.0028,(3.0/1080.0)
Total,1686.0,1077.0,0.0011,(3.0/2763.0)


In [5]:
print(spam_gbm2.predict(test))

gbm prediction progress: |████████████████████████████████████████████████| 100%


predict,p0,p1
1,1.26565e-14,1
1,1.58762e-13,1
1,2.22045e-16,1
1,0.727781,0.272219
1,1.37668e-14,1
1,9.34142e-13,1
0,0.996316,0.00368436
1,3.3995e-13,1
1,2.57172e-09,1
1,0,1
